In [1]:
# Get a list of devices like GPUs and CPUs available to TF

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15278802870524772955
]


In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
mnist

Datasets(train=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x0000027C23F7BF98>, validation=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x0000027C240580B8>, test=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x0000027C240580F0>)

In [4]:
mnist.train.images.shape

(55000, 784)

In [5]:
mnist.test.labels.shape

(10000, 10)

In [6]:
mnist.train.labels.shape

(55000, 10)

In [7]:
mnist.test.images.shape

(10000, 784)

In [46]:
import tensorflow as tf
#placeholders and variables declaration
x=tf.placeholder(tf.float32,[None,784])
w=tf.Variable(tf.zeros([784,10]))
b=tf.Variable(tf.zeros([10]))
y_original=tf.placeholder(tf.float32,[None,10])
y_pred=tf.nn.softmax(tf.matmul(x,w)+b)
cross_entropy=tf.reduce_mean(-tf.reduce_sum(y_original*tf.log(y_pred),reduction_indices=[1]))
#cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = y_original, labels = y_pred))
train_step=tf.train.GradientDescentOptimizer(0.05).minimize(cross_entropy)

In [47]:
#make a sesion
sess=tf.InteractiveSession()
#initializing vlues
tf.global_variables_initializer().run()

In [48]:
#training
for _ in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    #print(batch_xs.shape,batch_ys.shape)
    sess.run(train_step, feed_dict={x: batch_xs, y_original: batch_ys})
    #correct_prediction = tf.equal(tf.argmax(y_original,1), tf.argmax(y_pred,1)) #at each row taking the highest value axis=1 means
    #accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32)) #https://www.dotnetperls.com/cast-tensorflow
    #print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_original: mnist.test.labels}))

in the above we find w,b as they are variables by minimizing entropy changng the valyes of w,b 

In [49]:

# Returns the index with the largest value across axis of a tensor.
correct_prediction = tf.equal(tf.argmax(y_original,1), tf.argmax(y_pred,1)) #at each row taking the highest value axis=1 means
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32)) #https://www.dotnetperls.com/cast-tensorflow
print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_original: mnist.test.labels}))

0.8997


In [50]:
# https://gist.github.com/greydanus/f6eee59eaf1d90fcb3b534a25362cea4
# https://stackoverflow.com/a/14434334
import matplotlib.pyplot as plt
import numpy as np
import time

def plt_dynamic(x, y, y_1, ax, colors=['b']):
    ax.plot(x, y, 'b', label="Train Loss")
    ax.plot(x, y_1, 'r', label="Test Loss")
    if len(x)==1:
        plt.legend()
    fig.canvas.draw()

In [ ]:
# summarizing everything in single cell
#again itialiing so for understanding
x=tf.placeholder(tf.float32,[None,784])
w=tf.Variable(tf.zeros([784,10]))
b=tf.Variable(tf.zeros([10]))
y_original=tf.placeholder(tf.float32,[None,10])
y_pred=tf.nn.softmax(tf.matmul(x,w)+b)
sess=tf.InteractiveSession()
#initializing vlues
tf.global_variables_initializer().run()



training_epochs = 1000
batch_size = 100
#isplay_step = 1
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = y_pred, labels = y_original))
train_step = tf.train.GradientDescentOptimizer(0.05).minimize(cross_entropy)

fig,ax = plt.subplots(1,1)
ax.set_xlabel('epoch') ; ax.set_ylabel('Soft Max Cross Entropy loss')
xs, ytrs, ytes = [], [], []
for epoch in range(training_epochs):
        train_avg_cost = 0.
        test_avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            _, c = sess.run([train_step, cross_entropy], feed_dict={x: batch_xs, y_original: batch_ys})
            train_avg_cost += c / total_batch
            c = sess.run(cross_entropy, feed_dict={x: mnist.test.images, y_original: mnist.test.labels})
            test_avg_cost += c / total_batch

        xs.append(epoch)
        ytrs.append(train_avg_cost)
        ytes.append(test_avg_cost)
        plt_dynamic(xs, ytrs, ytes, ax)
        

plt_dynamic(xs, ytrs, ytes, ax)
correct_prediction = tf.equal(tf.argmax(y_original,1), tf.argmax(y_pred,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("Accuracy:", accuracy.eval({x: mnist.test.images, y_original: mnist.test.labels}))